# Monad Design pattern

Başlamadan önce bu design pattern daha ziyade fonksiyonel yaklaşımlar içindir. Biz genel olarak obje odaklı programlama mantığına yönelik sistemleri konuştuk. Bu mantığı rahatlıkla  class sistemine uygulamak mümkün. 

# Farkı monad yapıları

Maybe Monad:  Yapılan hesaplama bir değer döndürebilir ya da döndürmeyebilir". Belki internet üzerinden almak istediğin değerler (mesela bir api cevabı) gelmeyebilir ya da opsiyonel girdilere bağlı işlemler yapıyor olabilirsin. Bu durumları hata almadan yönetmek için faydalıdır.

State Monad:  Bir yapının durumunu içinde tutar ve fonksiyonlar arası geçişini sağlar. Bir sistemi modellemek için oldukça faydalıdır. 

Başka monadlar olsada bizim için faydalı olacaklar şimdilik bunlar

# Maybe Monad


Bir maybe monad sınıfı oluşturulalım

In [7]:
class Maybe:
    # başlangıçta bir değer alır
    def __init__(self, value):
        self._value = value

    # monad içindeki değeri fonksiyona sokmak için
    def bind(self, func):
        if self._value is None:
            return Maybe(None)
        else:
            return Maybe(func(self._value))
    # eğer içinde değer yoksa işlemez böylece hata alınmaz 

    # monad içi boş mu diye kontrol edip eğer boşsa bir default value vermeyi sağlar
    def orElse(self, default):
        if self._value is None:
            return Maybe(default)
        else:
            return self

    # değeri monaddan çıkartmak için. Eğer bir int işliyorsan ve 
    # işlemler bitince bir int istiyorsan 
    # monaddan böylece çıkarsın
    def unwrap(self):
        return self._value

    #dunder method tanımları

    # or operatörü ile düzgün çalışmak için 
    def __or__(self, other):
        return Maybe(self._value or other._value)

    # printte değerin gözükebilmesi için
    def __str__(self):
        if self._value is None:
            return 'Monad Değeri -> None'
        else:
            return 'Monad Değeri -> {}'.format(self._value)
    
    # bir list içinde değerin güzel gözükmesi için
    def __repr__(self):
        return str(self)

    # eşitlik operatörü ile düzgün çalışması için
    def __eq__(self, other):
        if isinstance(other, Maybe):
            return self._value == other._value
        else:
            return False

    # not equal operatörü ile düzgün çalışmak için
    def __ne__(self, other):
        return not (self == other)

    # değer none değilse true none ise false döner
    def __bool__(self):
        return self._value is not None

In [8]:

## monadlarımıza uygulamak için iki yeni fonksiyon tanımadık
## bu basit fonksiyonlar ile monadı nasıl kullanacağımızı göreceğiz

def add_one(x):
    return x + 1

def double(x):
    return x * 2

In [9]:

result = Maybe(3).bind(add_one).bind(double)
print(result)  # 8

result = Maybe(None).bind(add_one).bind(double)
print(result)  # None

result = Maybe(None).bind(add_one).bind(double).orElse(10)
print(result)  #  10

result = Maybe(None) | Maybe(1)
print(result) #  1


Monad Değeri -> 8
Monad Değeri -> None
Monad Değeri -> 10
Monad Değeri -> 1


# State Monad 

In [12]:
class State:
    def __init__(self, state):
        self.state = state

    # bu dunder metodu ile çağrılınca verilen tepkinin değişmesi sağlanır.
    def __call__(self, value):
        return (self.state[1], State((self.state[0] + 1, value)))
    # şuan bu class ile üretilen objeler çağrılınca kendi state bilgisine bir ekleyerek yeni bir obje oluşturur.
    # return olarak iki değer sağlar. Biri count değer ise yeni bir sayaç objesi
    

In [13]:
# bir obje oluşturarak monad yapısını kullanması sağlanır
counter = State((0, 0))

# for döngüsü ile yapının test edilmesi
for i in range(5):
    result, counter = counter(i)
    print(f"Computation result: {result}, count: {counter.state[0]}") 

Computation result: 0, count: 1
Computation result: 0, count: 2
Computation result: 1, count: 3
Computation result: 2, count: 4
Computation result: 3, count: 5
